In [ ]:
#define hostname
HOSTNAME = '192.168.1.100'

In [ ]:
from pyrpl import Pyrpl
p = Pyrpl(config="", # do not use a configfile
          hostname=HOSTNAME)

In [ ]:
p.rp.iq2.setup?

In [ ]:
iq = p.rp.iq2
print('Retrieved iq module "%s"' % iq.name)

# setup the iq module iq2 so that both demodulation quadratures are visible on the scope
iq.setup(input='in1',
         amplitude=0.5,
         output_direct='out1',
         output_signal='quadrature',
         frequency=2**14, # set the frequency to half the demodulation
         phase=0, #tune the phase as necessary
         modulation_at_2f='off',
         demodulation_at_2f='on',
         acbandwidth=500)

scope = p.rp.scope
scope.setup(input1='iq2', input2='iq2_2',
            duration=8,
            average=True,
            trigger_source='immediately',
            rolling_mode=True)

# now you can view the measurement on the scope

In [ ]:
# possibly tune the demodulation phase or quadrature_factor, or frequency
iq.phase = 90
iq.quadrature_factor = 100
iq.frequency -= 0.1
print(iq.frequency)

## Continuous low-level readout of the IQ accumulator magnitude and phase

In [ ]:
# How it works:
# When iq.frequency is written, the accumulator waits for iq._na_sleepcycles clock cycles (a 8 ns)
# and then averages over iq._na_averages clock cycles, the resulting sum being ready in iq._nadata_total
print(iq._na_sleepcycles, iq._na_averages, iq._nadata_total)

In [ ]:
#Example
from pyrpl.async_utils import sleep
import numpy as np


iq._na_sleepcycles = 0  # we need no sleep cycles, since there is no transient behaviour in our setup

avg_duration = 0.1  # average over 0.1 s
iq._na_averages = avg_duration / 8e-9  # set number of averages of accumulator

print("Sleep for", iq._na_sleepcycles, "cycles")
print("Average for", iq._na_averages, "cycles\n")

# make a local storage of iq.frquency, amplitude, _na_averages to save the 
# comminucation time needed to read those values all over again
frequency = iq.frequency
na_averages = iq._na_averages
amplitude = iq.amplitude

# Acquisition
print("Magnitude\t phase")
for i in range(5):  # acquisition loop, take 5 samples
    iq.frequency = frequency  # writing to the frequency register triggers the next averaging run 
    sleep(na_averages*8e-9)  # wait for averaging to finish
    signal = iq._nadata_total/na_averages  # result is stored in iq._nadata
    dB = 20*np.log10(signal/amplitude)  # conversion in dB w.r.t. drive amplitude
    mag = np.abs(signal)  # magnitude
    phase = np.angle(signal, deg=True)%360  # phase in degrees
    print("%.2e\t %.1f" %(mag, phase))